# 😊 Facial Emotion Recognition with FER2013
## Training High-Accuracy CNN Model using Transfer Learning

**What this notebook does:**
- Trains a deep learning model to detect 7 emotions from facial images
- Uses EfficientNet (transfer learning) for higher accuracy
- Saves the trained model as `face_emotionModel.h5`
- You can download the model and use it in your Flask app

**Emotions detected:** Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral

---

### 📋 Instructions for Beginners:
1. **Enable GPU:** Go to `Runtime` → `Change runtime type` → Select `T4 GPU` or `L4 GPU`
2. **Run cells in order:** Click the play button (▶️) on each cell from top to bottom
3. **Wait for training:** Training will take 30-60 minutes depending on GPU
4. **Download model:** At the end, you'll download `face_emotionModel.h5`

Let's begin! 🚀

## 🔧 Step 1: Setup and Installation

Installing required packages and mounting Google Drive.

In [ ]:
# Install required packages
!pip install -q tensorflow==2.15.0
!pip install -q kaggle
!pip install -q efficientnet

print("✅ Packages installed successfully!")

# Check GPU availability
import tensorflow as tf
print(f"\n🖥️  GPU Available: {tf.test.is_gpu_available()}")
print(f"GPU Device: {tf.test.gpu_device_name()}")
print(f"TensorFlow Version: {tf.__version__}")

In [ ]:
# Mount Google Drive to save model
from google.colab import drive
drive.mount('/content/drive')

# Create a folder for this project
!mkdir -p /content/drive/MyDrive/FER2013_Models
print("✅ Google Drive mounted!")

## 📦 Step 2: Download and Prepare Dataset

We'll use the FER2013 dataset from Kaggle.

### Option A: Using Kaggle API (Recommended)

**How to get your Kaggle API key:**
1. Go to https://www.kaggle.com/
2. Sign in to your account
3. Click on your profile picture → `Settings`
4. Scroll to `API` section → Click `Create New Token`
5. A file `kaggle.json` will download
6. Upload it in the next cell

In [ ]:
# Upload your kaggle.json file
from google.colab import files

print("📤 Please upload your kaggle.json file...")
uploaded = files.upload()

# Setup Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle API configured!")

In [ ]:
# Download FER2013 dataset from Kaggle
print("⏬ Downloading FER2013 dataset...")
!kaggle datasets download -d msambare/fer2013

# Unzip the dataset
print("📂 Extracting dataset...")
!unzip -q fer2013.zip -d /content/fer2013

print("✅ Dataset downloaded and extracted!")
print("\n📊 Dataset structure:")
!ls -la /content/fer2013/

### Verify Dataset

In [ ]:
import os

# Set dataset paths
TRAIN_DIR = '/content/fer2013/train'
TEST_DIR = '/content/fer2013/test'

# Check if dataset exists
if os.path.exists(TRAIN_DIR) and os.path.exists(TEST_DIR):
    print("✅ Dataset found!\n")
    
    # Count images per emotion
    emotions = os.listdir(TRAIN_DIR)
    print("Training data distribution:")
    for emotion in sorted(emotions):
        count = len(os.listdir(os.path.join(TRAIN_DIR, emotion)))
        print(f"  {emotion}: {count} images")
    
    print(f"\nTotal training images: {sum([len(os.listdir(os.path.join(TRAIN_DIR, e))) for e in emotions])}")
    print(f"Total test images: {sum([len(os.listdir(os.path.join(TEST_DIR, e))) for e in emotions])}")
else:
    print("❌ Dataset not found. Please check the previous steps.")

## 🎨 Step 3: Data Preprocessing and Augmentation

Preparing images and creating data generators with augmentation for better accuracy.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight

# Image parameters
IMG_SIZE = 224  # EfficientNet uses 224x224
BATCH_SIZE = 32  # Reduced for Colab memory
EPOCHS = 30

print("📊 Data Preprocessing Settings:")
print(f"  Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")

# Data augmentation for training (helps improve accuracy)
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,           # Normalize to [0, 1]
    rotation_range=15,           # Rotate images randomly
    width_shift_range=0.15,      # Shift horizontally
    height_shift_range=0.15,     # Shift vertically
    shear_range=0.15,            # Shear transformation
    zoom_range=0.15,             # Zoom in/out
    horizontal_flip=True,        # Flip horizontally
    fill_mode='nearest',         # Fill missing pixels
    validation_split=0.2         # Use 20% for validation
)

# Only rescaling for test data
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

print("\n✅ Data augmentation configured!")

In [ ]:
# Create data generators
print("📂 Creating data generators...\n")

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='rgb',  # EfficientNet uses RGB
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=False
)

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print(f"\n📋 Emotion Classes: {class_labels}")
print(f"\n✅ Training samples: {train_generator.samples}")
print(f"✅ Validation samples: {val_generator.samples}")
print(f"✅ Test samples: {test_generator.samples}")

In [ ]:
# Visualize some sample images
print("🖼️  Sample Training Images:\n")

sample_images, sample_labels = next(train_generator)

plt.figure(figsize=(15, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(sample_images[i])
    emotion_idx = np.argmax(sample_labels[i])
    plt.title(class_labels[emotion_idx])
    plt.axis('off')
plt.tight_layout()
plt.show()

print("✅ Data visualization complete!")

## 🧠 Step 4: Build the Model (Transfer Learning with EfficientNet)

Using EfficientNet pretrained on ImageNet for better accuracy.

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

print("🏗️  Building EfficientNet-based model...\n")

# Load EfficientNetB0 pretrained on ImageNet
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze base model initially
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(7, activation='softmax')  # 7 emotions
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Model built successfully!\n")
model.summary()

## 🎯 Step 5: Train the Model

Training in two phases:
1. **Phase 1:** Train with frozen base (faster)
2. **Phase 2:** Fine-tune with unfrozen base (higher accuracy)

In [ ]:
# Setup callbacks
checkpoint = ModelCheckpoint(
    '/content/best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=4,
    min_lr=1e-7,
    verbose=1
)

callbacks = [checkpoint, early_stop, reduce_lr]

print("✅ Training callbacks configured!")

In [ ]:
# Phase 1: Train with frozen base
print("\n" + "="*60)
print("🚀 PHASE 1: Training with frozen base model")
print("="*60 + "\n")

history_phase1 = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Phase 1 training complete!")

In [ ]:
# Phase 2: Fine-tune with unfrozen base
print("\n" + "="*60)
print("🎯 PHASE 2: Fine-tuning with unfrozen base model")
print("="*60 + "\n")

# Unfreeze the base model
base_model.trainable = True

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training
history_phase2 = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Phase 2 training complete!")
print("\n🎉 MODEL TRAINING FINISHED!")

## 📊 Step 6: Evaluate the Model

In [ ]:
# Plot training history
import matplotlib.pyplot as plt

# Combine both phases
acc = history_phase1.history['accuracy'] + history_phase2.history['accuracy']
val_acc = history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy']
loss = history_phase1.history['loss'] + history_phase2.history['loss']
val_loss = history_phase1.history['val_loss'] + history_phase2.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.axvline(x=15, color='r', linestyle='--', label='Fine-tuning starts')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.axvline(x=15, color='r', linestyle='--', label='Fine-tuning starts')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()

print("✅ Training curves plotted!")

In [ ]:
# Evaluate on test set
print("\n📊 Evaluating on test set...\n")

test_loss, test_accuracy = model.evaluate(test_generator)

print("\n" + "="*60)
print("FINAL TEST RESULTS")
print("="*60)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print("="*60)

# Get predictions
print("\n🔮 Generating predictions for confusion matrix...")
test_generator.reset()
predictions = model.predict(test_generator, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

print("✅ Predictions generated!")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, 
            yticklabels=class_labels)
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Classification report
print("\n📋 Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))

print("\n✅ Evaluation complete!")

## 💾 Step 7: Save the Model

In [ ]:
# Save to Google Drive
print("💾 Saving model to Google Drive...\n")

model.save('/content/drive/MyDrive/FER2013_Models/face_emotionModel.h5')
print("✅ Model saved to Google Drive: /content/drive/MyDrive/FER2013_Models/face_emotionModel.h5")

# Also save to local Colab for download
model.save('/content/face_emotionModel.h5')
print("✅ Model saved to local Colab: /content/face_emotionModel.h5")

print("\n🎉 MODEL SAVED SUCCESSFULLY!")
print("\nYou can now download it using the cell below.")

In [ ]:
# Download the model to your computer
from google.colab import files

print("📥 Downloading face_emotionModel.h5...\n")
files.download('/content/face_emotionModel.h5')

print("\n✅ Download started! Check your browser's download folder.")
print("\n📌 Next Steps:")
print("   1. Move the downloaded file to your FACE_DETECTION folder")
print("   2. Run your Flask app: python3 app.py")
print("   3. Open http://localhost:5000 in your browser")

## 🔮 Step 8: Test the Model with Predictions

In [ ]:
# Function to predict emotion from uploaded image
from tensorflow.keras.preprocessing import image
from google.colab import files
import cv2
from PIL import Image
import io

EMOTION_RESPONSES = {
    'angry': "You look angry. Take a deep breath! 😤",
    'disgust': "You seem disgusted. What's bothering you? 🤢",
    'fear': "You appear fearful. Don't worry! 😨",
    'happy': "You're smiling! Keep it up! 😊",
    'sad': "You are frowning. Why are you sad? 😢",
    'surprise': "You look surprised! 😮",
    'neutral': "You have a neutral expression. 😐"
}

def predict_emotion(img_path):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    emotion_idx = np.argmax(predictions[0])
    emotion = class_labels[emotion_idx]
    confidence = predictions[0][emotion_idx]
    
    # Display result
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predicted: {emotion.upper()} ({confidence*100:.1f}%)\n{EMOTION_RESPONSES[emotion]}", 
              fontsize=14, pad=20)
    plt.tight_layout()
    plt.show()
    
    return emotion, confidence

print("✅ Prediction function ready!")

In [ ]:
# Upload and test an image
print("📤 Upload an image to test emotion detection:\n")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\n🔮 Analyzing {filename}...\n")
    emotion, confidence = predict_emotion(filename)
    print(f"\n✅ Emotion: {emotion.upper()}")
    print(f"✅ Confidence: {confidence*100:.2f}%")
    print(f"✅ Message: {EMOTION_RESPONSES[emotion]}")

## 🚀 Step 9: Tips to Improve Accuracy

### Current Model Performance
Your model should achieve **60-70% accuracy** with the current setup.

### Ways to Increase Accuracy:

1. **Face Detection Preprocessing** (Best improvement: +5-10%)
   ```python
   # Use MTCNN or dlib to detect and crop faces before training
   # This removes background noise
   ```

2. **Larger Model** (+3-7%)
   - Use EfficientNetB3 or EfficientNetB4 instead of B0
   - Change: `EfficientNetB0` → `EfficientNetB3`

3. **More Training** (+2-5%)
   - Increase epochs: 30 → 50
   - Train longer with patience

4. **Class Balancing** (+2-4%)
   ```python
   # Add class weights to handle imbalanced data
   from sklearn.utils.class_weight import compute_class_weight
   class_weights = compute_class_weight('balanced', 
                                        classes=np.unique(train_generator.classes),
                                        y=train_generator.classes)
   # Use in model.fit: class_weight=dict(enumerate(class_weights))
   ```

5. **Ensemble Models** (+3-6%)
   - Train multiple models and average predictions
   - Combine EfficientNet + ResNet + VGG

6. **Test Time Augmentation (TTA)** (+1-3%)
   ```python
   # Make predictions on augmented versions and average
   ```

7. **Better Preprocessing**
   - Histogram equalization
   - Face alignment
   - Remove glasses, hats in preprocessing

8. **Mix of Datasets**
   - Combine FER2013 with RAF-DB or AffectNet
   - More diverse training data

### Expected Accuracy:
- **Basic CNN:** 50-60%
- **Current setup (EfficientNet):** 60-70%
- **With face detection:** 70-75%
- **With all optimizations:** 75-80%
- **State-of-the-art (research):** 80-90%

**Note:** FER2013 is challenging due to low resolution (48x48) and ambiguous labels.

## 🎉 Congratulations!

You've successfully:
- ✅ Downloaded and prepared the FER2013 dataset
- ✅ Built a CNN model with transfer learning (EfficientNet)
- ✅ Trained the model with data augmentation
- ✅ Evaluated model performance
- ✅ Saved the trained model as `face_emotionModel.h5`

### 📌 Next Steps:

1. **Download your model** (done above)
2. **Move it to your project:** Place `face_emotionModel.h5` in your `FACE_DETECTION` folder
3. **Install remaining packages locally:**
   ```bash
   pip3 install opencv-python --user
   ```
4. **Run your Flask app:**
   ```bash
   cd FACE_DETECTION
   python3 app.py
   ```
5. **Test locally:** Open http://localhost:5000
6. **Deploy to Render** (I'll guide you!)

### 💬 Questions?
Let me know if you need help with:
- Running the Flask app locally
- Deploying to Render
- Improving model accuracy
- Any troubleshooting

**You did great! 🚀**